In [34]:
#import des fichiers et load des ressources
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('2016_Building_Energy_Benchmarking.csv')
pd.options.display.max_columns = data.shape[1]

In [35]:
data.sample(1)

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
3130,40028,2016,NonResidential,Warehouse,Sandpoint #5,7561 63rd Ave NE,Seattle,WA,98115.0,0225049077,4,NORTHEAST,47.68454,-122.26235,1940,1.0,0,384772,0,384772,"Non-Refrigerated Warehouse, Office",Non-Refrigerated Warehouse,349953.0,Office,63872.0,NaN,NaN,NaN,69.0,36.700001,39.0,84.099998,86.5,15206762.0,16139557.0,0.0,2642108.0,9014872.0,61918.87891,6191888.0,False,NaN,Compliant,NaN,391.7,1.02


In [36]:
#variables cohérentes
cols = ['OSEBuildingID', 'DataYear', 'BuildingType',
       'PropertyName','Neighborhood','YearBuilt','NumberofBuildings','NumberofFloors',
        'PropertyGFATotal','PropertyGFAParking','PropertyGFABuilding(s)',
       'LargestPropertyUseType', 'ENERGYSTARScore',
       'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
        'Electricity(kBtu)','NaturalGas(kBtu)','TotalGHGEmissions']
data = data[cols]

In [37]:
data['BuildingType'].value_counts()

NonResidential          1460
Multifamily LR (1-4)    1018
Multifamily MR (5-9)     580
Multifamily HR (10+)     110
SPS-District K-12         98
Nonresidential COS        85
Campus                    24
Nonresidential WA          1
Name: BuildingType, dtype: int64

In [38]:
data_length_before = data.shape[0]
data['BuildingType'].replace({'Nonresidential COS': 'NonResidential',
                              'Nonresidential WA': 'NonResidential'}, inplace=True)
data = data.loc[data['BuildingType'] == 'NonResidential']

print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1830


In [39]:
#valeurs manquantes
data.isna().sum()

OSEBuildingID               0
DataYear                    0
BuildingType                0
PropertyName                0
Neighborhood                0
YearBuilt                   0
NumberofBuildings           2
NumberofFloors              0
PropertyGFATotal            0
PropertyGFAParking          0
PropertyGFABuilding(s)      0
LargestPropertyUseType      6
ENERGYSTARScore           540
SiteEnergyUseWN(kBtu)       3
SteamUse(kBtu)              2
Electricity(kBtu)           2
NaturalGas(kBtu)            2
TotalGHGEmissions           2
dtype: int64

In [40]:
#drop toutes lignes qui ont des valeurs nulles sauf EnergyScore
data_length_before = data.shape[0]
cols = ['LargestPropertyUseType', 'SiteEnergyUseWN(kBtu)', 'TotalGHGEmissions', 'NaturalGas(kBtu)', 'Electricity(kBtu)', 'SteamUse(kBtu)', 'NumberofBuildings']
data.dropna(subset=cols, inplace=True)
data.isna().sum()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  7


In [41]:
data.describe()

,OSEBuildingID,DataYear,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ENERGYSTARScore,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions
count,1539.000000,1539.0,1539.000000,1539.000000,1539.000000,1.539000e+03,1539.000000,1.539000e+03,1003.000000,1.539000e+03,1.539000e+03,1.539000e+03,1.539000e+03,1539.000000
mean,16609.779727,2016.0,1961.307342,1.035737,4.283951,1.120106e+05,13829.220273,9.818137e+04,63.790628,7.761522e+06,4.967574e+05,5.435044e+06,1.672264e+06,165.047498
std,13858.481096,0.0,32.888052,0.569622,6.796834,1.918960e+05,43730.728115,1.685664e+05,28.923639,1.884349e+07,5.314667e+06,1.305847e+07,5.513280e+06,571.657895
min,1.000000,2016.0,1900.000000,0.000000,0.000000,1.128500e+04,0.000000,3.636000e+03,1.000000,0.000000e+00,0.000000e+00,-1.154170e+05,0.000000e+00,-0.800000
25%,609.500000,2016.0,1929.000000,1.000000,1.000000,2.872800e+04,0.000000,2.768500e+04,44.000000,1.265929e+06,0.000000e+00,7.075250e+05,0.000000e+00,19.335000
50%,21213.000000,2016.0,1965.000000,1.000000,2.000000,4.721500e+04,0.000000,4.508200e+04,71.000000,2.739122e+06,0.000000e+00,1.633105e+06,4.313920e+05,48.350000
75%,24629.500000,2016.0,1988.000000,1.000000,4.000000,1.028880e+05,0.000000,9.290600e+04,89.000000,7.227405e+06,0.000000e+00,5.046944e+06,1.409886e+06,136.735000
max,50226.000000,2016.0,2015.000000,9.000000,99.000000,1.952220e+06,512608.000000,1.765970e+06,100.000000,2.966717e+08,1.349435e+08,2.745325e+08,1.381912e+08,12307.160000


In [42]:
data_length_before = data.shape[0]
cols = data.select_dtypes(include=[np.number]).columns
data = data[~(data[cols] < 0).any(axis=1)].copy()
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  1


In [43]:
data_length_before = data.shape[0]
difference = abs(data['PropertyGFATotal'] - (data['PropertyGFABuilding(s)'] + data['PropertyGFAParking']))
data = data[difference == 0]
print(f'lignes supprimés  {data_length_before - data.shape[0]}')

lignes supprimés  0


In [44]:
#création d'une colonne ancienneté à partir des variable DataYear et YearBuilt
data['Ancienneté'] = data['DataYear'] - data['YearBuilt']
data = data.drop('YearBuilt', axis=1)


In [45]:
data = pd.concat([data, pd.get_dummies(data['Neighborhood'], prefix='Neighborhood')], axis=1)
data = pd.concat([data, pd.get_dummies(data['LargestPropertyUseType'], prefix='LargestPropertyUseType')], axis=1)
dummy_cols = data.filter(regex='^(LargestPropertyUseType_|Neighborhood_)')
data = data.loc[:, (data != 0).any(axis=0) | (dummy_cols != 0).any(axis=0)]


In [46]:
#energy non nulle
data_energy_star_filtered = data[data['ENERGYSTARScore'].notna()]
data.to_csv('energy_star_only.csv', index=False)
data.loc[data['ENERGYSTARScore'].isnull(), 'ENERGYSTARScore'] = -1

In [47]:
data.to_csv('cleaned_data.csv', index=False)

In [48]:
data.shape[0]

1538

In [49]:
data

,OSEBuildingID,DataYear,BuildingType,PropertyName,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,ENERGYSTARScore,SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),TotalGHGEmissions,Ancienneté,Neighborhood_BALLARD,Neighborhood_Ballard,Neighborhood_CENTRAL,Neighborhood_Central,Neighborhood_DELRIDGE,...,LargestPropertyUseType_Other - Services,LargestPropertyUseType_Other - Utility,LargestPropertyUseType_Other/Specialty Hospital,LargestPropertyUseType_Parking,LargestPropertyUseType_Performing Arts,"LargestPropertyUseType_Personal Services (Health/Beauty, Dry Cleaning, etc)",LargestPropertyUseType_Police Station,LargestPropertyUseType_Pre-school/Daycare,LargestPropertyUseType_Prison/Incarceration,LargestPropertyUseType_Refrigerated Warehouse,"LargestPropertyUseType_Repair Services (Vehicle, Shoe, Locksmith, etc)",LargestPropertyUseType_Residence Hall/Dormitory,LargestPropertyUseType_Residential Care Facility,LargestPropertyUseType_Restaurant,LargestPropertyUseType_Retail Store,LargestPropertyUseType_Self-Storage Facility,LargestPropertyUseType_Senior Care Community,LargestPropertyUseType_Social/Meeting Hall,LargestPropertyUseType_Strip Mall,LargestPropertyUseType_Supermarket/Grocery Store,LargestPropertyUseType_Urgent Care/Clinic/Other Outpatient,LargestPropertyUseType_Wholesale Club/Supercenter,LargestPropertyUseType_Worship Facility
0,1,2016,NonResidential,Mayflower park hotel,DOWNTOWN,1.0,12,88434,0,88434,Hotel,60.0,7.456910e+06,2003882.00,3.946027e+06,1.276453e+06,249.98,89,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,2016,NonResidential,Paramount Hotel,DOWNTOWN,1.0,11,103566,15064,88502,Hotel,61.0,8.664479e+06,0.00,3.242851e+06,5.145082e+06,295.86,20,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,2016,NonResidential,5673-The Westin Seattle,DOWNTOWN,1.0,41,956110,196718,759392,Hotel,43.0,7.393711e+07,21566554.00,4.952666e+07,1.493800e+06,2089.28,47,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,2016,NonResidential,HOTEL MAX,DOWNTOWN,1.0,10,61320,0,61320,Hotel,56.0,6.946800e+06,2214446.25,2.768924e+06,1.811213e+06,286.43,90,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,8,2016,NonResidential,WARWICK SEATTLE HOTEL (ID8),DOWNTOWN,1.0,18,175580,62000,113580,Hotel,75.0,1.465650e+07,0.00,5.368607e+06,8.803998e+06,505.01,36,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,50222,2016,NonResidential,Horticulture building,GREATER DUWAMISH,1.0,1,12294,0,12294,Office,46.0,9.430032e+05,0.00,5.242709e+05,3.254750e+05,20.94,26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3372,50223,2016,NonResidential,International district/Chinatown CC,DOWNTOWN,1.0,1,16000,0,16000,Other - Recreation,-1.0,1.053706e+06,0.00,3.965461e+05,5.537300e+05,32.17,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3373,50224,2016,NonResidential,Queen Anne Pool,MAGNOLIA / QUEEN ANNE,1.0,1,13157,0,13157,Other - Recreation,-1.0,6.053764e+06,0.00,1.792159e+06,3.973739e+06,223.54,42,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3374,50225,2016,NonResidential,South Park Community Center,GREATER DUWAMISH,1.0,1,14101,0,14101,Other - Recreation,-1.0,7.828413e+05,0.00,3.488702e+05,3.706010e+05,22.11,27,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
